<a href="https://colab.research.google.com/github/iitmcvg/InterIIT_2021/blob/model_eval/model_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def model_eval(weights_path, img_augmentation):
  import tensorflow as tf
  from tensorflow import keras
  from tensorflow.keras import layers
  from tensorflow.keras.layers.experimental import preprocessing
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.applications import EfficientNetB0
  from tqdm.notebook import tqdm
  import os
  from PIL import Image
  import numpy as np

  output_dict = {}

  from google.colab import drive
  drive.mount('/content/drive')

  IMG_SIZE=224
  NUM_CLASSES=48

  preprocess_input = tf.keras.applications.efficientnet.preprocess_input
  rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

  inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
  x = img_augmentation(inputs)
  x = preprocess_input(x)
  x = base_model(x, training=False)
  x = global_average_layer(x)
  x = tf.keras.layers.Dropout(0.2)(x)
  outputs = prediction_layer(x)
  model = tf.keras.Model(inputs, outputs)
  base_learning_rate = 0.001
  model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                metrics=['accuracy'])
  
  base_model.trainable = True
  fine_tune_at = 150
  for layer in base_model.layers[:fine_tune_at]:
    layer.trainable =  False

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate/10),
              metrics=['accuracy'])
  
  model.load_weights(weights_path)

  test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "/content/drive/MyDrive/GTSRB/Online-Test-sort",
  seed=123,
  image_size=(IMG_SIZE, IMG_SIZE))

  y1 = np.array([]) 
  y2 = np.array([])
  for x, y in tqdm(test_dataset):
    y_true=np.argmax(model.predict(x),axis=1)
    y1=np.append(y1,y_true)
    y2=np.append(y2,y.numpy())

  from sklearn.metrics import accuracy_score

  output_dict.add("Test Accuracy:", accuracy_score(y1, y2)*100)

  import seaborn as sns
  import pandas as pd
  import matplotlib.pyplot as plt
  
  df_cm = pd.DataFrame(cf)
  plt.figure(figsize = (30,30))
  sns.heatmap(df_cm, annot=True)
  plt.savefig("cf_matrix.png")

  output_dict.add("Confusion Matrix file path:", "/content/cf_matrix.png")

  from sklearn.metrics import classification_report
  
  output_dict.add("Classification report:", classification_report(y1, y2))

  import heapq

  top5softmax = np.zeros((len(test_dataset), 5))
  i = 0
  for x, y in tqdm(test_dataset):
    top5softmax[i] = heapq.nlargest(5, range(NUM_CLASSES), model.predict(x).take)
    i = i+1

  import pandas as pd
  top5_dataframe = pd.DataFrame(top5softmax, columns = ['1', '2', '3', '4', '5'])
  confusion_matrix = pd.crosstab(top5_dataframe['1'], top5_dataframe['2'], rownames=['Best class'], colnames=['Second best class'])
  plt.figure(figsize = (30,30))
  sns.heatmap(confusion_matrix, annot=True)
  plt.savefig("Top_2_classes.png")

  output_dict.add("Top 2 classes matrix", "/content/Top_2_classes.png")

  return output_dict